In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [16]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from siamese.data import SiameseDataSource
from siamese.model import (
    resnet_baseline, ModelRunner, LossCallback, 
    FillingCallback, PairScoresCallback, F1Callback, 
    PrecisionCallback, RecallCallback, AccuracyCallback)

In [3]:
TRAIN_FOLDER = "/home/ivb/nvme/data/raw/train"
TRAIN_CSV = "/home/ivb/nvme/data/train.csv"

INFER_FOLDER = "/home/ivb/nvme/data/raw/test"

In [4]:
loaders = SiameseDataSource.prepare_loaders(
    mode="train",
    n_workers=8,
    batch_size=512,
    train_folder=TRAIN_FOLDER,
    train_csv=TRAIN_CSV,
    train_folds=[1, 2, 3, 4], 
    valid_folds=[5],
    #infer_folder=INFER_FOLDER,
)

Train samples: 11264
Train batches: 22
Valid samples: 2560
Valid batches: 5


In [5]:
model = resnet_baseline(resnet=dict(arch="resnet18", pooling="GlobalAvgPool2d"))
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, verbose=True)

In [17]:
import collections
from catalyst.dl.callbacks import (
    ClassificationLossCallback, 
    Logger, TensorboardLogger,
    OptimizerCallback, SchedulerCallback, CheckpointCallback, OneCycleLR)

# the only tricky part
n_epochs = 1000
logdir = "../siamese/logs"

callbacks = collections.OrderedDict()

callbacks["loss"] = LossCallback()
callbacks["optimizer"] = OptimizerCallback()

# OneCylce custom scheduler callback
# callbacks["scheduler"] = OneCycleLR(
#     cycle_len=n_epochs,
#     div=3, cut_div=4, momentum_range=(0.95, 0.85))

# Pytorch scheduler callback
callbacks["pair_scores"] = PairScoresCallback()
callbacks["valid_filling"] = FillingCallback()
callbacks["f1"] = F1Callback()
callbacks["precision"] = PrecisionCallback()
callbacks["recall"] = RecallCallback()
callbacks["accuracy"] = AccuracyCallback()
callbacks["saver"] = CheckpointCallback()
callbacks["logger"] = Logger()
callbacks["tflogger"] = TensorboardLogger()

In [7]:
runner = ModelRunner(
    model=model, 
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler)

In [ ]:
runner.train(
    loaders=loaders, 
    callbacks=callbacks, 
    logdir=logdir,
    epochs=5, start_epoch=610, verbose=True)




610 * Epoch (train):   0% 0/22 [00:00<?, ?it/s]


610 * Epoch (train):   0% 0/22 [00:17<?, ?it/s, accuracy_score=0.60688, base/batch_time=7.69111, base/data_time=7.65998, base/sample_per_second=66.57039, f1_score=0.03432, filling=0.02785, loss=0.64686, lr=0.00100, momentum=0.90000, precision_score=0.04334, recall_score=0.61628]


610 * Epoch (train):   5% 1/22 [00:17<06:13, 17.80s/it, accuracy_score=0.60688, base/batch_time=7.69111, base/data_time=7.65998, base/sample_per_second=66.57039, f1_score=0.03432, filling=0.02785, loss=0.64686, lr=0.00100, momentum=0.90000, precision_score=0.04334, recall_score=0.61628]


610 * Epoch (train):   5% 1/22 [00:22<06:13, 17.80s/it, accuracy_score=0.60688, base/batch_time=0.08891, base/data_time=0.08153, base/sample_per_second=5758.70288, f1_score=0.03430, filling=0.02785, loss=0.64554, lr=0.00100, momentum=0.90000, precision_score=0.04336, recall_score=0.61656]


610 * Epoch (train):   9% 2/22 [00:22<04:39, 13.99s/it, accuracy_score=0.60688, bas